### **线性回归简洁实现**

通过使用深度学习框架来简洁的实现**线性回归**模型生成数据集

#### **生成数据集**

##### **导入所需包**

In [1]:
import torch 
import numpy as np  
from torch.utils import data  
from d2l import torch as d2l  

##### **构造真实 $w$，$b$、及其他数据**

In [3]:
true_w = torch.tensor([2,-3.4])         # 给定真实w，b
true_b = 4.2  
features, labels = d2l.synthetic_data(true_w, true_b, 1000)         # 通过synthetic_data人工数据合成函数生成features和labels

---

#### **读取数据**

##### **调用框架中现有的API来读取数据**

In [6]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)       # 使用DataLoader函数每次随机从(dataset中挑选出batch_size个样本，并随机打乱顺序)

batch_size = 10         
data_iter = load_array((features, labels), batch_size)          # 构造iter

next(iter(data_iter))       # 把data_iter转成python的iter，通过next函数得到x，y

[tensor([[-0.4656,  0.0443],
         [-0.4651,  0.2159],
         [ 1.6327, -0.5312],
         [ 0.2651,  0.9108],
         [ 1.3119, -0.1822],
         [ 0.2633, -0.7431],
         [ 1.2022,  0.8866],
         [ 0.0839, -0.3997],
         [-1.8440, -0.2656],
         [-0.6579,  0.7857]]),
 tensor([[3.1194],
         [2.5300],
         [9.2873],
         [1.6358],
         [7.4366],
         [7.2506],
         [3.5953],
         [5.7096],
         [1.4162],
         [0.2307]])]

---

#### **定义模型**

##### **使用框架的预定义好的层**

In [7]:
# 'nn' 是神经网络的缩写
from torch import nn        # Neural Networks的缩写等价于全连接层

net = nn.Sequential(nn.Linear(2, 1))   # 指定输入的维度是2，输出的维度是1；可以直接使用Linear层(单层神经网络)，把这个层放到一个有序容器中(Sequential)

##### **初始化模型参数**

In [8]:
net[0].weight.data.normal_(0, 0.01)  # network可以通过0访问到Linear，通过.weight访问到w，通过.data访问真实data，使用正态分布替换掉data中的值
net[0].bias.data.fill_(0)   #bias指偏差，将数据设为0 

tensor([0.])

##### **计算均方误差使用的是MSELoss类，也称平方范数**

In [9]:
loss = nn.MSELoss()         # MSELoss可以直接调用

##### **实例化SGD(随机梯度下降)实例**

In [10]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)       # SGD要求需要传入至少两个参数：①所有net.parameters包括w和b  ②学习率lr
# optim是指optimize优化，sgd是优化的一种方法

---

#### **训练过程**

In [11]:
num_epochs = 3          # 迭代3个周期
for epoch in range(num_epochs):         # 对每一次迭代扫一遍数据
    for X, y in data_iter:          # 从data_iter中把X，y拿出来
        l = loss(net(X), y)         # 放进net里面，此处net()中自带模型参数，所以不需要把参数写进去，得到预测值y和真实y做loss
        trainer.zero_grad()         # trainer优化器把梯度清零
        l.backward()            # 计算backward，这里pytorch以及计算过sum就不需要再写
        trainer.step()          # 调用step函数更新模型
    l = loss(net(features), labels)         # 扫完一遍数据之后，将所有的features放到network中，和labels做loss
    print(f'epoch {epoch + 1}, loss {l:f}')     

epoch 1, loss 0.000154
epoch 2, loss 0.000097
epoch 3, loss 0.000098


---

#### **对比结果**

下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。 要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。 正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [13]:
w = net[0].weight.data  
print('w的估计误差为:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data  
print('b的估计误差为:',true_b - b)

w的估计误差为: tensor([ 0.0004, -0.0012])
b的估计误差为: tensor([0.0004])
